In [ ]:
#!pip install tqdm > /dev/null 2>&1
#!pip install ipywidgets > /dev/null 2>&1
#!gdown -V > /dev/null 2>&1
#!gdown --folder url -O /content/data > /dev/null 2>&1

# Import necessary libraries and functions
import os
import shutil
import warnings
from time import time
import zipfile

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from tqdm.notebook import tqdm
from scipy.stats import norm

from sklearn.model_selection import (StratifiedKFold, cross_val_score, GridSearchCV,
                                     train_test_split)
from sklearn.metrics import (balanced_accuracy_score, classification_report,
                             matthews_corrcoef, confusion_matrix, mean_squared_error,
                             roc_curve, auc, roc_auc_score, accuracy_score,
                             f1_score, precision_score)
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier

from mlxtend.plotting import plot_decision_regions


# Copy Datasets
source_file_path = '/content/drive/MyDrive/PhD/1-DEDProject/8-ifocus-dataset/find biomarker in the metabolomics dataset/classification/datasets/data.zip'
destination_file_path = '/content/data.zip'
shutil.copy(source_file_path, destination_file_path)

# Uzip the zipped dataset file
zip_file_path = '/content/data.zip'
extract_to_path = '/content/data/'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

In [ ]:
# Suppress all warnings
warnings.filterwarnings('ignore')

############################################################################################# Plots
# Function to plot histograms for different performance metrics
def plot_histograms(auc_list, test_acc_list, mcc_list, bal_acc_list, model_name, method_name):
    metrics = [auc_list, test_acc_list, mcc_list, bal_acc_list]
    metric_names = ["AUC", "Test Accuracy", "MCC", "Balanced Accuracy"]

    plt.figure(figsize=(15, 10))

    # iterate over the metrics and plot histogram
    for i, metric in enumerate(metrics):
        plt.subplot(2, 2, i + 1)  # 2 rows, 2 columns
        plt.hist(metric, bins=20, edgecolor='black', alpha=0.7)
        plt.title(metric_names[i])
        plt.xlabel('Value')
        plt.ylabel('Frequency')

    plt.tight_layout()

    # Constructing the filename and saving the figure
    directory = "/content/report/plots"
    filename = os.path.join(directory, f"{model_name}_{method_name}.png")

    plt.savefig(filename, dpi=300)
    plt.close()

# Function to plot AUC with error bars for different models and methods
def plot_auc_with_error_bars(results_df):
    # Extracting relevant data for plotting
    labels = results_df["Model"] + "-" + results_df["Feature Reduction Method"]
    auc_means = results_df["AUC Mean"].values
    auc_stds = results_df["AUC Std"].values

    # Setting the figure size and creating the bar plot with error bars
    plt.figure(figsize=(14, 8))
    plt.bar(range(len(labels)), auc_means, yerr=auc_stds, align='center', alpha=0.7, ecolor='black', capsize=10)
    plt.ylabel('AUC')
    plt.xticks(range(len(labels)), labels, rotation=45, ha="right")

    # Annotations for better clarity
    for i, v in enumerate(auc_means):
        plt.text(i, v + 0.01, f"{v:.2f}", ha='center', va='bottom', fontsize=8, color='black')

    plt.tight_layout()  # Adjust layout
    plt.title('AUC with Error Bars for Different ML Model-FR Method Combinations', y=1.05)  # Adjust title position

    # Saving the plot
    plt.savefig("/content/report/auc_barplot_with_error_bars.png", bbox_inches='tight')
    plt.close()  # Close the figure after saving to free up memory    